In [0]:
import pandas as pd
from datetime import datetime

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
df1=spark.table('sw_hot_table').toPandas()
df2=spark.table('Market').toPandas()
merged=pd.merge(df1,df2,on='Date',how='inner')

In [0]:
# Compute daily return
df1=df1.sort_values('Date').reset_index(drop=True)
df2=df2.sort_values('Date').reset_index(drop=True)
df1['Stock_Return'] = df1['Close_Price'].pct_change() * 100
df2['Market_Return'] = df2['Sensex_Close'].pct_change() * 100

In [0]:
df1=df1[['Date','Close_Price','Stock_Return']]
df2=df2[['Date','Sensex_Close','Market_Return']]
merged_data=pd.merge(df1,df2,on='Date',how='inner')
three_year_covariance_matrix=merged_data[['Stock_Return','Market_Return']].cov()
Beta_3_Years=three_year_covariance_matrix.iloc[0,1]/three_year_covariance_matrix.iloc[1,1]


#latest_traded_date=df2['Date'].iloc[-1]
latest_traded_date=df2['Date'].max()
one_year_covariance_matrix_filter=merged_data[merged_data['Date']>=latest_traded_date - pd.DateOffset(months=12)]
one_year_covariance_matrix=one_year_covariance_matrix_filter[['Stock_Return','Market_Return']].cov()
Beta_1_Year=one_year_covariance_matrix.iloc[0,1]/one_year_covariance_matrix.iloc[1,1]
print(Beta_3_Years,Beta_1_Year)

In [0]:
#CAPM
Rf=6
Market_Return_1_Year = (one_year_covariance_matrix_filter['Market_Return'].mean()) * 252
expected_rate_of_return_1_Year= Rf + (Beta_1_Year*(Market_Return_1_Year-Rf))
print(expected_rate_of_return_1_Year.round(3))
#print(((merged['Close_Price'].iloc[-1]/merged['Close_Price'].iloc[0] -1)*100).round(3))
print(((one_year_covariance_matrix_filter['Close_Price'].iloc[-1]/one_year_covariance_matrix_filter['Close_Price'].iloc[0] -1)*100).round(3))

In [0]:
#CAPM 3 Years
Rf=6
Market_Return_3_Years = (merged_data['Market_Return'].mean())*252
expected_rate_of_return_3_Year=Rf+(Beta_3_Years*(Market_Return_3_Years-Rf))
print(expected_rate_of_return_3_Year.round(3))
print(((df1['Close_Price'].iloc[-1]/df1['Close_Price'].iloc[0] -1)*100).round(3))

In [0]:
%sql
--CREATE OR REPLACE TABLE CAPM_Metrics (
  --Metric_Date DATE,
  --Beta_3_Years DOUBLE,
  --CAPM_3_Years DOUBLE,
  --Actual_3_Years_Returns DOUBLE,
  --Beta_1_Year DOUBLE,
  --CAPM_1_Year DOUBLE,
  --Actual_1_Year_Return DOUBLE
--) USING delta

In [0]:
capm_metrics = pd.DataFrame({
    'Metric_Date': [datetime.now().date()],
    'Beta_1_Year': [(one_year_covariance_matrix.iloc[0,1]/one_year_covariance_matrix.iloc[1,1]).round(3)],
    'CAPM_3_Years': [expected_rate_of_return_3_Year.round(3)],
    'Actual_3_Years_Returns': [((df1['Close_Price'].iloc[-1] / df1['Close_Price'].iloc[0] - 1) * 100).round(3)],
    'CAPM_1_Year': [expected_rate_of_return_1_Year.round(3)],
    'Beta_3_Years':[(three_year_covariance_matrix.iloc[0,1]/three_year_covariance_matrix.iloc[1,1]).round(3)],
    'Actual_1_Year_Return': [((one_year_covariance_matrix_filter['Close_Price'].iloc[-1] / one_year_covariance_matrix_filter['Close_Price'].iloc[0] - 1) * 100).round(3)]
})

capm_metrics_spark = spark.createDataFrame(capm_metrics)
capm_metrics_spark.createOrReplaceTempView("updates")

In [0]:
spark.sql("""
MERGE INTO CAPM_Metrics AS target
USING updates AS source
ON target.Metric_Date = source.Metric_Date
WHEN MATCHED THEN UPDATE SET
  target.Beta_3_Years = source.Beta_3_Years,
  target.CAPM_3_Years = source.CAPM_3_Years,
  target.Actual_3_Years_Returns = source.Actual_3_Years_Returns,
  target.Beta_1_Year = source.Beta_1_Year,
  target.CAPM_1_Year = source.CAPM_1_Year,
  target.Actual_1_Year_Return = source.Actual_1_Year_Return
WHEN NOT MATCHED THEN
INSERT (Metric_Date,Beta_3_Years, CAPM_3_Years, Actual_3_Years_Returns,Beta_1_Year, CAPM_1_Year, Actual_1_Year_Return)
VALUES (source.Metric_Date, source.Beta_3_Years,source.CAPM_3_Years, source.Actual_3_Years_Returns,source.Beta_1_Year, source.CAPM_1_Year, source.Actual_1_Year_Return)
""")

In [0]:
%sql
select * 
from CAPM_Metrics

In [0]:
#Calculate Market Returns for the past 1 year
#Latest_Market_Price=df2['Sensex_Close'].iloc[-1]
#Latest_Traded_Date=df2['Date'].iloc[-1]
#print(Latest_Traded_Date,Latest_Market_Price)

In [0]:
#Latest_Stock_Price=df1['Close_Price'].iloc[-1]
#One_Year_Ago_Stock_Price=df1[df1['Date']<=One_Year_Ago_Date].iloc[-1]
#One_Year_Ago_Stock_Price=One_Year_Ago_Stock_Price['Close_Price']
#One_Year_Ago_Stock_Return=((Latest_Stock_Price/One_Year_Ago_Stock_Price -1)*100).round(3)
#print(One_Year_Ago_Stock_Return)

In [0]:
%sql
--select Sensex_Close
--from Market
--where Date='2024-08-02'